In [ ]:
from pathlib import Path
import sys
from IPython.display import clear_output
import warnings
# Filter user warnings from sklearn/lgbm to keep output clean
warnings.filterwarnings("ignore", category=UserWarning)

p = Path.cwd()
# Robust root detection: prefer the ancestor that contains full_data/training
root = None
for candidate in [p] + list(p.parents):
    if (candidate / "full_data" / "training").exists():
        root = candidate
        break
# Fallback: previous heuristics + scan descendants (handles papermill using parent cwd)
if root is None:
    root = p
    while root.parent != root and not (root / "pyproject.toml").exists():
        root = root.parent
    # If not found upwards, search downwards a few levels for pyproject
    if not (root / "pyproject.toml").exists():
        candidates = list(p.rglob('pyproject.toml'))
        print('pyproject candidates (rglob):', candidates)
        if candidates:
            root = candidates[0].parent
    # If not found upwards, search downwards a few levels for pyproject (duplicate check)
    if not (root / "pyproject.toml").exists():
        candidates = list(p.rglob('pyproject.toml'))
        if candidates:
            root = candidates[0].parent
# Ensure the project root and the `full_data` folder are on sys.path
sys.path.insert(0, str(root))
full_data_path = root / "full_data"
if full_data_path.exists():
    sys.path.insert(0, str(full_data_path))
import os
print("Project root set to:", root)
print("Added to sys.path:", sys.path[0])
print('cwd at runtime:', os.getcwd())
# Print any papermill-related environment values to help identify the notebook path
for k, v in os.environ.items():
    if k.startswith('PAPERMILL') or 'NOTEBOOK' in k.upper() or k in ('PWD', 'PYTHONPATH'):
        print(k, v)

Project root set to: d:\Programming\Python\comfyui-image-scorer\full_data


In [ ]:
import os
import sys
print('Diagnostics — cwd:', os.getcwd())
print('Diagnostics — sys.path[0:3]:', sys.path[:3])
import importlib
try:
    import training.data_utils
    import training.run
    importlib.reload(training.data_utils)
    importlib.reload(training.run)
    from training.run import train_model, optimize_hyperparameters
    print("Modules reloaded.")
except Exception as e:
    print('Import error while loading training modules:', type(e), e)
    raise

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
import importlib
import random
import numpy as np

# Reload shared.config to ensure detailed clean state (fix for AutoSaveDict recursion bug)
import shared.config

importlib.reload(shared.config)
from shared.config import config

# Reload training.run to use the new config object
import training.run
# Reload data_utils to pick up new interaction logic
import training.data_utils 
import training.config_utils

importlib.reload(training.run)
importlib.reload(training.data_utils)
importlib.reload(training.config_utils)

from training.run import (
    LivePlotCallback,
    prepare_optimization_setup,
    generate_combos,
    evaluate_hyperparameter_combo,
    optimize_hyperparameters
)
from training.helpers import resolve_path
# Import granular steps for explicit pipeline
from training.data_utils import (
    load_training_data, 
    filter_unused_features, 
    add_interaction_features
)

# Set output path for live graph
config["training"]["live_graph_path"] = "training/output/graph_hpo.png"
live_plot_path = resolve_path(config["training"]["live_graph_path"])
print(f"Live plot will be saved to: {live_plot_path}")

Live plot will be saved to: c:\ComfyUI\trainer\training\output\graph_hpo.png


In [ ]:
# Pre-load and Process Data Explicitly
vectors_path = resolve_path(config["vectors_file"])
scores_path = resolve_path(config["scores_file"])

print("--- Step 1: Loading Data ---")
X, y = load_training_data(vectors_path, scores_path)
print(f"Loaded Data Shape: {X.shape}")

print("\n--- Step 2: Filtering Unused Features ---")
# Removes features with zero variance or zero importance in a quick probe
X, kept_indices = filter_unused_features(X, y)
print(f"Filtered Data Shape: {X.shape}")

print("\n--- Step 3: Generating Interaction Features ---")
# Adds top 500 polynomial interactions (feature_A * feature_B)
X, _ = add_interaction_features(X, y, target_k=200)
print(f"Final Data Shape (with Interactions): {X.shape}")
print("Data Preparation Complete.")

--- Step 1: Loading Data ---
Loaded Data Shape: (6495, 2322)

--- Step 2: Filtering Unused Features ---
Filtering features... Initial shape: (6495, 2322)
Loading filtered data from cache: C:\ComfyUI\trainer\training\output\filtered_data_cache.npz
Filtered Data Shape: (6495, 1357)

--- Step 3: Generating Interaction Features ---
Loading interaction data from cache: C:\ComfyUI\trainer\training\output\interaction_data_cache.npz
Final Data Shape (with Interactions): (6495, 1857)
Data Preparation Complete.


### Optimization Pipeline Confirmation

You asked: *"Are you doing all 3 optimizations before starting with the hyperparameters?"*

**Answer:**
1.  **Filtering**: Yes, applied in Step 2 above.
2.  **Interaction Features**: Yes, applied in Step 3 above.
3.  **Target Transformation (Yeo-Johnson)**: **Yes**, but this is applied **during training** (inside the `run.py` module). 
    *   Since the transformation acts on the target `y`, it's best integrated into the `train_model` function so it can be properly inverse-transformed for predictions and error calculation on the fly.
    *   It is currently active for **every** model trained in the loop below.

In [ ]:
from training.config_utils import crossover_config, generate_random_config

# Setup Loop Variables
max_iters = config["training"]["max_iters"]
max_combos = config["training"]["max_combos"]

# Initialize current_cfg
current_cfg = generate_random_config()
print(f"Initialized HPO loop with max_iters={max_iters}, max_combos={max_combos}")

Initialized HPO loop with max_iters=100, max_combos=3


In [ ]:
import importlib
import training.run
importlib.reload(training.run)
from training.run import optimize_hyperparameters
print("training.run reloaded with TransformedTargetRegressor fix.")

training.run reloaded with TransformedTargetRegressor fix.


In [ ]:
# Optimization Loop
for i in range(max_iters):
    clear_output(wait=True)
    # Refresh references
    top_cfg = config["training"]["top"]
    fastest_cfg = config["training"]["fastest"]
    slowest_cfg = config["training"]["slowest"]
    
    max_combos = config["training"]["max_combos"]

    # Strategy selection
    rand_val = random.random()
    if rand_val < 0.05:
        base_cfg = current_cfg
        strategy = "RANDOM_START"
    elif rand_val < 0.25:
        base_cfg = fastest_cfg
        strategy = "FASTEST"
    elif rand_val < 0.60:
        base_cfg = top_cfg
        strategy = "TOP"
    elif rand_val < 0.85:
        # Crossover
        candidates = [
            c for c in [top_cfg, fastest_cfg, slowest_cfg] if c["best_score"] > -9999
        ]
        if len(candidates) < 2:
            candidates = [c for c in [top_cfg, fastest_cfg, current_cfg] if c]
            if len(candidates) < 2:
                candidates = [generate_random_config(), generate_random_config()]

        parents = random.sample(candidates, 2)
        base_cfg = crossover_config(dict(parents[0]), dict(parents[1]))
        strategy = "CROSSOVER"
    else:
        base_cfg = slowest_cfg
        strategy = "SLOWEST"
        # max_combos = 1

    print(f"\nIter {i + 1}/{max_iters} — Strategy: {strategy}")

    # optimize_hyperparameters handles updates and saving internally now.
    results = optimize_hyperparameters(
        base_cfg=base_cfg, max_combos=max_combos, X=X, y=y, strategy=strategy
    )

    # Info only loop
    for candidate_cfg, metrics in results:
        r2 = metrics["r2"]
        t_time = metrics["training_time"]

        # Check if this result matches current bests (just for log/info)
        is_top = r2 == top_cfg["best_score"]
        is_fastest = t_time == fastest_cfg["training_time"]

        if is_top:
            print(f"  [Info] This batch produced the current TOP score: {r2:.6f}")
        if is_fastest:
            print(
                f"  [Info] This batch produced the current FASTEST time: {t_time:.4f}s"
            )

    # Refresh local current_cfg for next RANDOM_START or Crossover inheritance
    if results:
        # Simply take the last one as 'current' for random drift
        current_cfg = results[-1][0]

print(f"\nFinished optimization.")
print(f"Top R2: {top_cfg['best_score']:.6f}")
print(
    f"Fastest Time: {fastest_cfg['training_time']:.6f}s (R2: {fastest_cfg['best_score']:.6f})"
)
print(
    f"Slowest (High Score) Time: {slowest_cfg['training_time']:.6f}s (R2: {slowest_cfg['best_score']:.6f})"
)


Iter 33/100 — Strategy: TOP
Optimizing hyperparameters over grid: {'learning_rate': [0.3221966168924104, 0.007819515626402838, 0.006397785512511412], 'n_estimators': [1980, 1800, 1620], 'num_leaves': [569, 518, 466], 'max_depth': [100, 94, 84], 'min_child_samples': [158, 144, 129], 'reg_alpha': [9.695187078391255, 0.5102285769113253, 0.41745974474562975], 'reg_lambda': [2.837802523458497, 2.5798204758713608, 2.3218384282842246], 'subsample': [0.9233412332969095, 0.7554610090611077, 0.41281112572356593], 'colsample_bytree': [0.3228352225904187, 0.2934865659912897, 0.26413790939216075], 'min_split_gain': [0.40859769615825575, 0.37145245105295976, 0.3343072059476638], 'early_stopping_rounds': [191, 48, 39]}
Evaluating hyperparameter combo 1/3, with params: {'learning_rate': 0.007819515626402838, 'n_estimators': 1620, 'num_leaves': 518, 'max_depth': 94, 'min_child_samples': 129, 'reg_alpha': 0.5102285769113253, 'reg_lambda': 2.837802523458497, 'subsample': 0.41281112572356593, 'colsample_

Training LightGBM: 100%|██████████| 1620/1620 [00:29<00:00, 55.37it/s]


r2=0.4193, time=29.2571s for Evaluated params {'learning_rate': 0.007819515626402838, 'n_estimators': 1620, 'num_leaves': 518, 'max_depth': 94, 'min_child_samples': 129, 'reg_alpha': 0.5102285769113253, 'reg_lambda': 2.837802523458497, 'subsample': 0.41281112572356593, 'colsample_bytree': 0.2934865659912897, 'min_split_gain': 0.3343072059476638, 'early_stopping_rounds': 48}
Evaluating hyperparameter combo 2/3, with params: {'learning_rate': 0.007819515626402838, 'n_estimators': 1620, 'num_leaves': 569, 'max_depth': 84, 'min_child_samples': 144, 'reg_alpha': 0.5102285769113253, 'reg_lambda': 2.3218384282842246, 'subsample': 0.9233412332969095, 'colsample_bytree': 0.3228352225904187, 'min_split_gain': 0.3343072059476638, 'early_stopping_rounds': 191}


Training LightGBM: 100%|██████████| 1620/1620 [00:27<00:00, 58.36it/s]


r2=0.4202, time=27.7584s for Evaluated params {'learning_rate': 0.007819515626402838, 'n_estimators': 1620, 'num_leaves': 569, 'max_depth': 84, 'min_child_samples': 144, 'reg_alpha': 0.5102285769113253, 'reg_lambda': 2.3218384282842246, 'subsample': 0.9233412332969095, 'colsample_bytree': 0.3228352225904187, 'min_split_gain': 0.3343072059476638, 'early_stopping_rounds': 191}
Evaluating hyperparameter combo 3/3, with params: {'learning_rate': 0.007819515626402838, 'n_estimators': 1800, 'num_leaves': 569, 'max_depth': 100, 'min_child_samples': 144, 'reg_alpha': 0.41745974474562975, 'reg_lambda': 2.5798204758713608, 'subsample': 0.9233412332969095, 'colsample_bytree': 0.2934865659912897, 'min_split_gain': 0.37145245105295976, 'early_stopping_rounds': 191}


Training LightGBM:  26%|██▋       | 476/1800 [00:09<00:23, 56.05it/s]